In [1]:
#   ▄████████    ▄████████ ████████▄  ████████▄   ▄█      ███             ▄████████    ▄████████  ▄████████    ▄█    █▄     ▄█   ▄█    █▄     ▄████████    ▄████████
#   ███    ███   ███    ███ ███   ▀███ ███   ▀███ ███  ▀█████████▄        ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ███    █▀  ███    ███ ███    ███ ███▌    ▀███▀▀██        ███    ███   ███    ███ ███    █▀    ███    ███   ███▌ ███    ███   ███    █▀    ███    ███
#  ▄███▄▄▄▄██▀  ▄███▄▄▄     ███    ███ ███    ███ ███▌     ███   ▀        ███    ███  ▄███▄▄▄▄██▀ ███         ▄███▄▄▄▄███▄▄ ███▌ ███    ███  ▄███▄▄▄      ▄███▄▄▄▄██▀
# ▀▀███▀▀▀▀▀   ▀▀███▀▀▀     ███    ███ ███    ███ ███▌     ███          ▀███████████ ▀▀███▀▀▀▀▀   ███        ▀▀███▀▀▀▀███▀  ███▌ ███    ███ ▀▀███▀▀▀     ▀▀███▀▀▀▀▀
# ▀███████████   ███    █▄  ███    ███ ███    ███ ███      ███            ███    ███ ▀███████████ ███    █▄    ███    ███   ███  ███    ███   ███    █▄  ▀███████████
#   ███    ███   ███    ███ ███   ▄███ ███   ▄███ ███      ███            ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ██████████ ████████▀  ████████▀  █▀      ▄████▀          ███    █▀    ███    ███ ████████▀    ███    █▀    █▀    ▀██████▀    ██████████   ███    ███
#   ███    ███                                                                         ███    ███                                                          ███    ███

# Reddit Archiver - An easy way to archive and view your Reddit Account. 

Downloads your profile info and posts/comments from your input reddit account. Creates a presentable webpage of your arhcived profile that can be saved locally. 

* Allows user to input desired Reddit Username and output directory for archive file via command line
* Downloads user data using BeautifulSoup.
* Saves data to JSON file
* Loads JSON File into HTML webpage and saves webpage to desired directory


By [dandex200](https://github.com/dandex200/) on Github


#### Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
from tkinter import filedialog, Tk

## User Input

In [2]:
#Username Input
user_input = input("Enter desired Reddit Username to Archive:")

#Output Directory Input using Tkinter
confirm_input = input("Choose Archive File Output Directory? (Y/N)")

#Select Output Directory if chosen
if confirm_input.lower() == 'y':
    root = Tk() 
    root.withdraw()
    output_dir = filedialog.askdirectory()
    root.destroy()
    print("Will save archive file to: %s" % output_dir)
else:
    print("No output directory selected - will save to current directory.")
    

Will save archive file to: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver


## Scraping

In [3]:
user_url = 'https://old.reddit.com/user/'+user_input

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(user_url, headers=headers)

soup = BeautifulSoup(response.content, 'html.parser')

title_check = soup.title.text.split(' ')[0]

if title_check == "overview":
    print("working")
else:
    print("User does not exist")

working


### Profile Info

In [4]:
username = soup.title.text.split(' ')[2]

comment_karma_box = soup.find('span', {'class': 'comment-karma'})
comment_karma = comment_karma_box.text if comment_karma_box is not None else ''

post_karma = soup.find('span', {'class': 'karma'})
post_karma = post_karma.text if post_karma is not None else ''


user_creation_date = soup.find('span', {'class': 'age'})
user_creation_date = user_creation_date.find('time')['datetime']

num_posts = 0
num_comments = 0

print(username)
print(comment_karma)
print(post_karma)
print(user_creation_date)
print(num_posts)
print(num_comments)

CardinalCreator
688
848
2018-12-11T23:36:45+00:00
0
0


### User Posts & Comments

In [6]:
comments = []
comment_count = 0
for comment in soup.find_all('div', class_='comment'):
    utc_date = comment.find('time', class_="live-timestamp")['datetime']
    score = comment.find('span', class_='unvoted').text
    subreddit = comment.find('a', class_="subreddit").text
    permalink = comment.find('a', class_="bylink")['href']
    text = comment.find('div', class_='md-container').text
    comment_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'text': text
    }
    comments.append(comment_data)
    comment_count += 1
print(comments)
print(comment_count)

[{'utc_date': '2023-02-12T06:46:54+00:00', 'score': '1 point', 'subreddit': 'ProductManagement', 'permalink': 'https://old.reddit.com/r/ProductManagement/comments/10zyoze/are_there_jobs_like_product_manager_in_the/j87pbw1/', 'text': 'I have experience working for a federal contractor on user-facing apps. The project I was on had product managers on both the contractor side and the government side. I was pleasantly surprised by how forward thinking the govt PMs were and open to change. Across agencies, there’s a fairly big push to modernize (some are doing better at that than others though). I enjoyed my stint doing govt work and would recommend to anyone\n\n'}, {'utc_date': '2023-01-31T03:31:54+00:00', 'score': '2 points', 'subreddit': 'Atlanta', 'permalink': 'https://old.reddit.com/r/Atlanta/comments/10pgjlc/aquarium_plants/j6ldltj/', 'text': "Southern Aquatics is my go-to local fish store (http://www.southernaquaticslfs.com/), but I've had solid luck with live plants from the Howell 

In [7]:
post_count = 0
for post in soup.find_all('div', class_='link'):
    print(post)
    post_count += 1
print(post_count)

<div class="thing id-t3_136wlyc linkflair odd link" data-author="CardinalCreator" data-author-fullname="t2_1pbmr1" data-comments-count="88" data-context="listing" data-domain="wsbtv.com" data-fullname="t3_136wlyc" data-gildings="0" data-is-gallery="false" data-nsfw="false" data-num-crossposts="0" data-oc="false" data-permalink="/r/Atlanta/comments/136wlyc/say_goodbye_to_the_potholes_on_atlantas_dekalb_ave/" data-promoted="false" data-rank="" data-score="166" data-spoiler="false" data-subreddit="Atlanta" data-subreddit-fullname="t5_2qiq9" data-subreddit-prefixed="r/Atlanta" data-subreddit-type="public" data-timestamp="1683141384000" data-type="link" data-url="https://www.wsbtv.com/news/local/atlanta/say-goodbye-potholes-atlantas-dekalb-ave/OATFUQ22WRG6NH54TSE3OPXRUE/" data-whitelist-status="all_ads" id="thing_t3_136wlyc" onclick="click_thing(this)"><p class="parent"></p><span class="rank"></span><div class="midcol unvoted"><div aria-label="upvote" class="arrow up login-required access-r

In [10]:
#Identify Posts

posts = []
post_count = 0

utc_date = 0
post_type = ""
subreddit = ""
post_title = ""
file_path = ""
permalink = ""
score = 0
text = ""

#self post
for self_post in soup.find_all('div', class_='self'):
    post_count += 1
    utc_date = self_post.find('time', class_="live-timestamp")['datetime']
    post_type = "self"
    subreddit = self_post.find('a', class_="subreddit").text
    post_title = self_post.find('a', class_="title").text
    score = self_post.find('div', {'class': 'score unvoted'}).text
    permalink = self_post.find('a', class_="bylink")['href']

    if self_post.find('div', {'class': 'expando-button'}): #If expando button present
        #Download permalink
        print("self expando")
        response = requests.get(permalink, headers={'User-Agent': 'Mozilla/5.0'})
        post_soup = BeautifulSoup(response.content, 'html.parser')
        #Extract text
        text = post_soup.find('div', {'class': 'entry'}).find('div', {'class': 'md-container'}).get_text()
    else:
        text = None

    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
    
#crosspost
for cross_post in soup.find_all('div', {'class':'link', 'data-crosspost-root-title': True}):
    post_count += 1
    utc_date = cross_post.find('time', class_="live-timestamp")['datetime']
    post_type = "crosspost"
    subreddit = cross_post.find('a', class_="subreddit").text
    post_title = cross_post.find('a', class_="title").text
    score = cross_post.find('div', {'class': 'score unvoted'}).text
    permalink = cross_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
#link
domains = ['old.reddit.com', 'i.redd.it', 'v.redd.it', 'i.imgur.com']
for link_post in soup.find_all(lambda tag: tag.has_attr('data-domain') and not any(domain in tag['data-domain'] for domain in domains )and 'self' not in tag.get('class', [])):
    post_count += 1
    utc_date = link_post.find('time', class_="live-timestamp")['datetime']
    post_type = "link"
    subreddit = link_post.find('a', class_="subreddit").text
    post_title = link_post.find('a', class_="title").text
    score = link_post.find('div', {'class': 'score unvoted'}).text
    permalink = link_post.find('a', class_="bylink")['href']
    
    #If expando, request permalink page, extract text, else text = NaN
    
    if link_post.find('div', {'class': 'expando-button'}): #If expando button present
        #Download permalink
        response = requests.get(permalink, headers={'User-Agent': 'Mozilla/5.0'})
        post_soup = BeautifulSoup(response.content, 'html.parser')
        #Extract text
        text = post_soup.find('div', {'class': 'entry'}).find('div', {'class': 'md-container'}).get_text()
    else:
        text = None
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
#video
for video_post in soup.find_all('div', {'class':'link', "data-domain": "v.redd.it"}):
    post_count += 1
    utc_date = video_post.find('time', class_="live-timestamp")['datetime']
    post_type = "video"
    subreddit = video_post.find('a', class_="subreddit").text
    post_title = video_post.find('a', class_="title").text
    score = video_post.find('div', {'class': 'score unvoted'}).text
    permalink = video_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
#picture 
for pic_post in soup.find_all('div', {'class':'link', "data-domain": "old.reddit.com"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)

for pic_post in soup.find_all('div', {'class':'link', "data-domain": "i.redd.it"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)

for pic_post in soup.find_all('div', {'class':'link', "data-domain": "i.imgur.com"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
print(post_count)

self expando
self expando
self expando
9


In [11]:
#Test if 25 Posts identified (Number of each page)
if (post_count + comment_count) != 25:
    print("Error")
else:
    print("working")

working


In [12]:
num_posts = len(posts)
num_comments = len(comments)

In [13]:
user_data = {
    'user_info': {
        'username': username,
        'user_creation_date': user_creation_date,
        'num_posts': num_posts,
        'num_comments': num_comments,
        'karma': {
            'post_karma': post_karma,
            'comment_karma': comment_karma
        }
    },
    'posts': posts,
    'comments': comments
}

In [14]:
with open('Temp_Files/user_data.json', 'w') as f:
    json.dump(user_data, f)

In [15]:
with open('Temp_Files/user_data.json') as f:
    data = json.load(f)

print(json.dumps(data, indent=4))


{
    "user_info": {
        "username": "CardinalCreator",
        "user_creation_date": "2018-12-11T23:36:45+00:00",
        "num_posts": 9,
        "num_comments": 16,
        "karma": {
            "post_karma": "848",
            "comment_karma": "688"
        }
    },
    "posts": [
        {
            "utc_date": "2022-10-20T16:59:02+00:00",
            "score": "36",
            "subreddit": "r/gatech",
            "permalink": "https://old.reddit.com/r/gatech/comments/y93mou/georgia_tech_to_break_ground_on_tech_squares/",
            "title": "Georgia Tech to break ground on Tech Square's Phase 3 this week",
            "type": "self",
            "text": "https://atlanta.urbanize.city/post/georgia-tech-square-development-breaking-ground-midtown-west-peachtree?fbclid=IwAR0R2ZLbk0u1Zl71dH9vRMG7pt9K5bKPYlxtVYlCI6WkAzLlai8raMwhXyQ\n\n"
        },
        {
            "utc_date": "2022-08-04T16:11:23+00:00",
            "score": "2",
            "subreddit": "r/Cardinals",
    

**To-DO: PT1**
* Successfully append comments, user info, and posts of first page to JSON in correct format
* Save pictures/videos to directory
* Present JSON and saved files in beginner format HTML webpage
* Apply saving to all pages of reddit account
* Add tests to ensure data is all saved (meets 25 posts per page, etc)
* Create Readme on Github
* Publish first time

**TO-DO PT2**
* Convert from notebook to script
* Create GUI where you can choose if you want to save pics/videos, input user/output directory in GUI, etc
* Apply ability to save pics/videos as choice
* Create Pretty looking HTML archive page
* Promote on Reddit
* Release as second update 
* Add ability to keep link in comment

**TO-DO PT3**
* blank

**Minor Additions**
* Accounting for pinned posts
* Profile pictures (snoovatar??)
* Trophies
* Add over 18 bypass for old.reddit